# S6. App Structure

## Алгоритм

1. На входе данные за 1 час - показания по экологии и деперсонализированные события по авто
2. Каждый автомобиль получает или теряет баллы в зависимости от показаний экодатчиков в момент проезда
3. Математика начисления баллов (и выставления порогов) модернизируется с каждой итерацией
4. Авто, чьи баллы превысили порог, передаются на обработку экополиции, те возвращаются к нам с информацией о том, какая достигнута точность
5. Повторяем пока не получим нужную точность

## 1. Получение входных данных

1. Отсортировать данные по времени
2. Распределить по локациям
3. Сформировать временные фреймы
4. Упаковать по временным фреймам данные по авто и экологии


In [16]:
import pandas as pd
from functools import reduce

eco_df = pd.read_csv("ecodata2.csv")
car_df = pd.read_csv("cardata2.csv")

eco_df.sort_values(by=['time'], inplace=True)
car_df.sort_values(by=['time'], inplace=True)

In [17]:
eco_df.head(2)

,id,city_id,location_id,camera_id,co,no2,so2,o3,pm25,pm10,temp,hum,time,ver,lat,lon,created
4999,18,2,741,4553,4.0,0.68,0,0.81,40,45,18.24,21.28,12/10/22 17:41,-1,43.237604,76.934758,12/10/22 11:41
4858,15,2,2,420,8.8,0.00,0,0.87,35,38,19.20,19.29,12/10/22 17:41,-1,43.238362,76.889989,12/10/22 11:41


In [18]:
time_from = pd.to_datetime('12/10/22 17:46')
time_to = pd.to_datetime('12/10/22 17:47')

eco_df.loc[(pd.to_datetime(eco_df['time']) >= time_from) & (pd.to_datetime(eco_df['time']) < time_to) & 
           (eco_df['camera_id'] == 420)]
car_df.loc[(pd.to_datetime(car_df['time']) >= time_from) & (pd.to_datetime(car_df['time']) < time_to) & 
           (car_df['camera_id'] == 420)]

,id,city_id,location_id,camera_id,time,lat,lon,lane,speed
3537,542d38f9-7c62-7646-19fe-6102ac3f2705,2,2,420,12/10/2022 17:46,43.238362,76.889989,2,28
3545,08388e77-06c9-43f2-3fe5-843b05174639,2,2,420,12/10/2022 17:46,43.238362,76.889989,2,26
3528,9a9e520b-d6af-b61f-b1bf-34657469dba9,2,2,420,12/10/2022 17:46,43.238362,76.889989,2,27
3520,bad78d4f-6efe-b277-6e34-b5d2e0c95feb,2,2,420,12/10/2022 17:46,43.238362,76.889989,2,38
3573,58ba91e7-d9d3-7df3-76c8-7eccdb8f1b32,2,2,420,12/10/2022 17:46,43.238362,76.889989,1,8
3563,0cc0c95b-db12-44a8-da70-b52253795640,2,2,420,12/10/2022 17:46,43.238362,76.889989,1,13
3554,933d7ca3-d008-5cd4-9c9d-2060175300f1,2,2,420,12/10/2022 17:46,43.238362,76.889989,1,1
3474,72e2b36b-c5f2-2307-67a8-76da5eb50f33,2,2,420,12/10/2022 17:46,43.238362,76.889989,3,29
3456,66aa4c08-badc-bf86-8ef7-b3c681a6c173,2,2,420,12/10/2022 17:46,43.238362,76.889989,1,33
3582,5839cfb2-270f-66e7-c203-71594097af32,2,2,420,12/10/2022 17:46,43.238362,76.889989,1,3


In [19]:
def create_time_frame(dataframes: list[dict], camera_id: str = None, time_from: str = None, time_to: str = None):
    df = pd.concat(dataframes)
    if camera_id is None:
        camera_ids = set(df['camera_id'])
        response = {}
        for camera_id in camera_ids:
            response[camera_id] = create_time_frame(
                dataframes=[dataframes[0], dataframes[1]],
                camera_id=camera_id,
                time_from=time_from,
                time_to=time_to
            )
        yield response
    else:
        if time_from is None or time_to is None:
            for time in sorted(
                set(pd.to_datetime(df['time']))
            ):
                time_from = time
                time_to = time + pd.Timedelta(minutes=1)
                new_eco_df = dataframes[0].loc[
                    (pd.to_datetime(dataframes[0]['time']) >= time_from) & 
                    (pd.to_datetime(dataframes[0]['time']) < time_to) & 
                    (dataframes[0]['camera_id'] == camera_id)
                ]
                new_car_df = dataframes[1].loc[
                    (pd.to_datetime(dataframes[1]['time']) >= time_from) & 
                    (pd.to_datetime(dataframes[1]['time']) < time_to) & 
                    (dataframes[1]['camera_id'] == camera_id)
                ]

                yield {time: [new_eco_df, new_car_df]}
        else:
            for time in sorted(
                set(pd.to_datetime(df['time']))
            ):
                time_from = time
                time_to = time + pd.Timedelta(minutes=1)
                if time <= time_to:
                    new_eco_df = dataframes[0].loc[
                        (pd.to_datetime(dataframes[0]['time']) >= time_from) & 
                        (pd.to_datetime(dataframes[0]['time']) < time_to) & 
                        (dataframes[0]['camera_id'] == camera_id)
                    ]
                    new_car_df = dataframes[1].loc[
                        (pd.to_datetime(dataframes[1]['time']) >= time_from) & 
                        (pd.to_datetime(dataframes[1]['time']) < time_to) & 
                        (dataframes[1]['camera_id'] == camera_id)
                    ]

                    yield {time: [new_eco_df, new_car_df]}


In [20]:
timeframe_420 = create_time_frame([eco_df, car_df], 420)
timeframe = create_time_frame([eco_df, car_df])

## 2. Начисление баллов

1. Сформировать хранилище для баллов
2. Подготовить функцию для начисления баллов
3. Прогнать входные данные через функцию начисления баллов по каждой локации фрейм за фреймом

In [21]:
import matplotlib.pyplot as plt
import math

In [22]:
car_ids = car_df['id'].unique()
score = {}
for car_id in car_ids:
    score[car_id] = 0
unexpected_timeframe = {}

In [23]:
for tf in timeframe:
    for cam, times in tf.items():
        pm25_avg_list = []
        pm25_list = []
        time_list = []
        time_dict = {}
        for time in times:
            for time, dataframes in time.items():
                if len(dataframes[0]):
                    time_list.append(time)
                    pm25 = dataframes[0]['pm25']
                    pm25_list.extend(pm25)
                    pm25_avg_list.append(sum(pm25) / len(pm25))
                    time_dict[time] = max(pm25)
        unexpected_timeframe[cam] = time_dict
        pm25_avg = sum(pm25_avg_list) / len(pm25_avg_list)
        pm25_sq = 0
        for pm25 in pm25_list:
            pm25_sq += (pm25 - pm25_avg) ** 2
        pm25_sq = math.sqrt(pm25_sq / len(pm25_list))
        pm25_expect = pm25_avg + pm25_sq

In [24]:
for cam, times in unexpected_timeframe.items():
    for time, pm25 in times.items():
        if pm25 >= pm25_expect:
            bad_timeframe = car_df.loc[
                (pd.to_datetime(car_df['time']) == time) &
                (car_df['camera_id'] == cam)
            ]
            ids = bad_timeframe['id']
            for id_ in ids:
                score[id_] += 1

## 3. Сохранение результата

Сохранить итоговое значение score в файл формата .csv

In [25]:
df = pd.DataFrame.from_dict(score, orient='index')

In [26]:
df.to_csv('score.csv')